 - mistral-instruct inference tutorial https://huggingface.co/blog/Andyrasika/mistral-7b-empowering-conversation
 - https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers bitsandbytes peft datasets accelerate trl --use-deprecated=legacy-resolver

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 12.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

from transformers import AutoTokenizer
import numpy as np

In [ ]:
device = "cpu"

In [ ]:
def load_checkpoint_model(path):
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(
        path,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto", # "cuda" to large, "cpu" somehow does not put LinearFP4 Layer on device
    )

    return model

base_model = "mistralai/Mistral-7B-Instruct-v0.2"
model = load_checkpoint_model(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [ ]:
def generate_text(model, instruction: str, max_new_tokens=1000) -> str:
  messages = [
      {"role": "user", "content": instruction},
  ]

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt") # wraps text with special tokens depending on role (assitant or user)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(model_inputs, max_new_tokens=max_new_tokens, do_sample=True)
  decoded = tokenizer.decode(generated_ids[0][len(model_inputs[0]):], skip_special_tokens=True) # [0]: first dimension has size 1
  return decoded


response = generate_text(model, "Generate an xml file that defines an arena and and places robots in it. Place 5 robots in the arena. The robots should form a circle with one robot at the center.\n\nDo not explain your results, only give the output.")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1665: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
print(response)

```xml
<arena>
  <bounds x="0" y="0" width="100" height="100"/>
  <robots>
    <robot id="0" x="50" y="50">
      <circle centerX="50" centerY="50" radius="5"/>
      <positionType type="fixed"/>
    </robot>
    <robot id="1" x="60" y="55">
      <circle centerX="50" centerY="50" radius="5"/>
    </robot>
    <robot id="2" x="40" y="55">
      <circle centerX="50" centerY="50" radius="5"/>
    </robot>
    <robot id="3" x="45" y="50">
      <circle centerX="50" centerY="50" radius="5"/>
    </robot>
    <robot id="4" x="55" y="50">
      <circle centerX="50" centerY="50" radius="5"/>
    </robot>
  </robots>
</arena>
```
